# Advanced Quantum Circle Walk Algorithm

## Introduction
This notebook provides a detailed explanation and implementation of the Quantum Circle Walk algorithm. We will cover the advanced theory behind the algorithm, show the implementation in Python using the `classiq` library, and visualize the results.

## Task:
The goal of this quantum program is to simulate a quantum random walk on a line topology with 16 nodes.
In a line topology, the nodes are arranged sequentially, and each node is connected to its immediate neighbors. For example, consider a line of 16 nodes numbered from 0 to 15. In this walk, the walker would move from its current node to an adjacent node (either left or right) with equal probability.
![Alt text](image.png)

## Advanced Theory
Quantum walks are the quantum analog of classical random walks. They leverage the principles of quantum superposition and interference, allowing quantum walks to explore graphs more efficiently than classical random walks. Quantum walks are crucial in developing quantum algorithms that can outperform classical algorithms.

### Types of Quantum Walks
1. **Discrete-Time Quantum Walks (DTQWs)**: Involve a quantum system (the walker) moving on a discrete set of positions (nodes) according to a unitary evolution operator. The walker's state is described by a quantum superposition of these positions.
2. **Continuous-Time Quantum Walks (CTQWs)**: Defined on a continuous time scale, where the evolution of the walk is governed by the Hamiltonian of the system. The probability distribution of the walker's position evolves continuously over time.

### Quantum Circle Walk Algorithm
The quantum circle walk algorithm is a specific type of DTQW where the walker moves on a circular graph. This algorithm leverages quantum superposition and interference to explore the graph efficiently.


#### Key Components
1. **Initialization**: Prepare the walker in an initial quantum state, often a uniform superposition of all possible positions.
2. **Walk Operator (W)**: Evolves the walker's state based on the adjacency matrix of the graph, ensuring that the walker can move to adjacent nodes.
3. **Shift Operator (S)**: Conditionally swaps the walker's position based on specific conditions, such as the presence of edges in the graph.
4. **Measurement**: Measure the quantum state of the walker to obtain the probability distribution of its position.

### Detailed Steps
1. **Initialization**: The walker is initialized in a uniform superposition state using a Hadamard transform. This step ensures that the walker starts in an equal superposition of all possible positions on the graph.
2. **Walk Operator (W)**: Applied iteratively to evolve the walker's state. It uses unitary operations to adjust the probability amplitudes of the walker's position, mimicking the adjacency matrix of the circular graph.
3. **Shift Operator (S)**: Conditionally swaps the walker's position based on the presence of edges in the graph. This operator uses controlled operations to update the walker's position according to the graph's structure.
4. **Measurement**: After applying the walk and shift operators for a specified number of steps, the quantum state of the walker is measured. This measurement collapses the superposition state into a specific position, and the probability distribution of the walker's position can be analyzed.


In [ ]:
import matplotlib
matplotlib.use('Agg')  # Set the backend to Agg for plotting

from classiq import QArray, X, H, qfunc, Output, QBit, allocate, create_model, show, synthesize, hadamard_transform, control, within_apply, inplace_prepare_state, repeat, SWAP, QNum
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Define the size of the qubits for the quantum circle walk
size = 4  # 4 qubits for 16 nodes

# Prepare the initial minus state
@qfunc
def prepare_minus_state(qbit: QBit):
    X(qbit)
    H(qbit)

# Oracle for diffusion
@qfunc
def diffusion_oracle(aux: QBit, x: QNum):
    aux ^= (x != 0)

# Apply the diffuser to the zero state
@qfunc
def apply_zero_diffuser(x: QNum):
    aux = QBit('aux')
    allocate(1, aux)
    within_apply(compute=lambda: prepare_minus_state(aux),
                 action=lambda: diffusion_oracle(aux, x))

# Iteration step for the W operator
def w_iteration(i: int, vertices: QNum, adjacent_vertices: QNum):
    prob = [0.0] * (2 ** size)
    if i > 0 and i < 2 ** size - 1:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    elif i == 0:
        prob[i + 1] = 0.5
        prob[2 ** size - 1] = 0.5
    elif i == 2 ** size - 1:
        prob[i - 1] = 0.5
        prob[0] = 0.5
    print(f'State={i}, prob vec={prob}')

    control(ctrl=vertices == i,
            operand=lambda: within_apply(
                compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
                action=lambda: apply_zero_diffuser(adjacent_vertices)))

# W operator for the quantum walk
@qfunc
def w_operator(vertices: QNum, adjacent_vertices: QNum):
    for i in range(2 ** size):
        w_iteration(i, vertices, adjacent_vertices)

# Oracle to check edges
@qfunc
def edge_check_oracle(result: Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    result |= (((vertices + adjacent_vertices) % 2) == 1)

# Swap bits between arrays
@qfunc
def bitwise_swap(x: QArray[QBit], y: QArray[QBit]):
    repeat(count=x.len,
           iteration=lambda i: SWAP(x[i], y[i]))

# S operator for the quantum walk
@qfunc
def s_operator(vertices: QNum, adjacent_vertices: QNum):
    result = QBit('result')
    edge_check_oracle(result, vertices, adjacent_vertices)
    control(ctrl=result == 1,
            operand=lambda: bitwise_swap(vertices, adjacent_vertices))

# Main function to execute the quantum circle walk
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    allocate(size, vertices)
    hadamard_transform(vertices)
    allocate(size, adjacent_vertices)
    w_operator(vertices, adjacent_vertices)
    s_operator(vertices, adjacent_vertices)

# Create the quantum model, synthesize, and show the quantum program
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)


### The implanted circuit:
![Alt text](image-1.png)
![Alt text](image-2.png)
![Alt text](image-3.png)

## Extensions:
Below we do some visualization and is not needed

In [ ]:
# Theoretical state probabilities based on the W operator iterations
def calculate_theoretical_probabilities(size, steps):
    num_states = 2 ** size
    probabilities = np.zeros(num_states)
    probabilities[0] = 1.0  # Start in state 0
    all_probabilities = [probabilities.copy()]
    for step in range(steps):
        new_probabilities = np.zeros(num_states)
        for j in range(num_states):
            if j > 0 and j < num_states - 1:
                new_probabilities[j - 1] += probabilities[j] * 0.5
                new_probabilities[j + 1] += probabilities[j] * 0.5
            elif j == 0:
                new_probabilities[j + 1] += probabilities[j] * 0.5
                new_probabilities[num_states - 1] += probabilities[j] * 0.5
            elif j == num_states - 1:
                new_probabilities[j - 1] += probabilities[j] * 0.5
                new_probabilities[0] += probabilities[j] * 0.5
        probabilities = new_probabilities
        all_probabilities.append(probabilities.copy())
    return all_probabilities


In [ ]:
# Visualization function
def visualize_state_probabilities(all_probabilities):
    steps = len(all_probabilities)
    num_states = len(all_probabilities[0])

    plt.figure(figsize=(10, 6))
    for step in range(steps):
        plt.plot(range(num_states), all_probabilities[step], label=f'Step {step}')

    plt.xlabel('States')
    plt.ylabel('Probabilities')
    plt.title('Quantum Circle Walk State Probabilities Over Steps')
    plt.legend()
    plt.savefig('quantum_circle_walk_probabilities.png')
    # Note: plt.show() may not work in some environments due to the 'Agg' backend
    plt.show()

# Calculate the theoretical probabilities over multiple steps
steps = 10  # Number of steps to simulate
all_probabilities = calculate_theoretical_probabilities(size, steps)

# Visualize the state probabilities
visualize_state_probabilities(all_probabilities)


## Conclusion
In this notebook, we have implemented and visualized the Quantum Circle Walk algorithm. We explained the advanced theory behind quantum walks, demonstrated the implementation, and visualized the probability distributions at different steps of the walk.

---

### Saving and Running the Notebook
Save this notebook as `quantum_circle_walk.ipynb`. You can run it in any Jupyter Notebook environment, ensuring you have the required libraries (`classiq`, `matplotlib`, `numpy`) installed.

To run the notebook:
1. Open Jupyter Notebook.
2. Navigate to the directory containing `quantum_circle_walk.ipynb`.
3. Open the notebook and run all cells.

This notebook provides a comprehensive explanation and visualization of the quantum circle walk algorithm.
